In [1]:
# Setting up google drive 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks')

Mounted at /content/gdrive


In [2]:
import my_utils as mu
import torch
from torch import nn

# The Task

* Our goal for this week is to write some code to create an MLP and then experiment with several training options and hyper-parameters.

* For this reason we will use again the same pipeline we used last week! 
    
* The Learning Outcome: Hands-on application of PyTorch's API for creating and training MLPs for classification.

In [3]:
# Read training and test data
batch_size = 256
train_iter, test_iter = mu.load_data_fashion_mnist(batch_size)
# type(train_iter)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [4]:
X,y = next(iter(train_iter))
print(X.shape)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([256, 1, 28, 28])


In [5]:
patches = X.unfold(2, 14, 14).unfold(3, 14, 14)
#patches = patches.reshape(256, 1, -1, 7, 7)

patches.shape
#patches[0]

torch.Size([256, 1, 2, 2, 14, 14])

In [6]:
def tuple_of_tensors_to_tensor(tuple_of_tensors):
    return  torch.stack(list(tuple_of_tensors), dim=0)

In [7]:
X.shape
patches.shape
def patching(batch):
  final = []
  batch = batch.unfold(2, 14, 14).unfold(3, 14, 14)
  for x in batch:
    p = x.flatten()
    p = torch.split(p,196)
    p = tuple_of_tensors_to_tensor(p)
    p.shape
    final.append(p)
  final = torch.stack(final)
  return final

X2 = patching(X)
X2.shape
#X2[0].mean(axis=1)

torch.Size([256, 4, 196])

In [8]:
#X2[0]

In [9]:
lin = nn.Linear(196, 10, bias=False)
testt = lin(X2)
parameters = []
for param in lin.parameters():
  parameters.append(param)
parameters = torch.stack(parameters)
parameters

tensor([[[ 0.0411, -0.0504,  0.0335,  ..., -0.0690,  0.0442,  0.0652],
         [ 0.0206,  0.0581,  0.0378,  ..., -0.0419, -0.0141,  0.0120],
         [-0.0490,  0.0388, -0.0500,  ...,  0.0350,  0.0054,  0.0182],
         ...,
         [-0.0197, -0.0322,  0.0258,  ..., -0.0199, -0.0204,  0.0116],
         [-0.0602, -0.0172,  0.0442,  ...,  0.0521,  0.0395, -0.0129],
         [ 0.0238,  0.0320, -0.0478,  ..., -0.0285, -0.0222, -0.0361]]],
       grad_fn=<StackBackward0>)

In [10]:
#testt

In [11]:
#torch.bmm(X2, parameters)

In [12]:
len(patches[0].flatten())
p2 = patches[0].flatten()
p2 = torch.split(p2,196)
p2 = tuple_of_tensors_to_tensor(p2)
p2.shape

torch.Size([4, 196])

In [13]:
from PIL import Image
import numpy as np
#Image.fromarray(torch.squeeze(patches[0][0]).detach().cpu().numpy())
p= torch.squeeze(patches[1][0][0][0]).detach().cpu().numpy().astype(np.uint8) * 255
print(np.min(p))
print(np.max(p))
Image.fromarray(p)

0
0


## Task 1


* The model below implements Softmax Regression. It has 1 Linear (or Fully-Connected Layer). Expand the `Net` class to have one more linear layer as follows: the 1st Linear layer will output `num_hidden` outputs. Set `num_hidden=10`.  Moreover, apply a `ReLU` activation function to the output of the 1st Linear layer. The second linear layer will have `num_hidden` inputs and 10 outputs (i.e. equal to the number of classes).



In [14]:
class Net(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(Net, self).__init__()
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        num_hidden = 196
        self.Linear1 = nn.Linear(num_inputs, num_hidden)
        self.relu = nn.ReLU()
        self.elu = nn.ELU()
        self.Linear2 = nn.Linear(196, 98)
        self.Linear3 = nn.Linear(98, 49)
        self.Linear4 = nn.Linear(4, 50)
        self.Linear5 = nn.Linear(50, 100)
        self.Linear6 = nn.Linear(100,25)
        self.Linear7 = nn.Linear(25,10)
        #self.Linear5 = nn.Linear(29, num_outputs)
    
    def tuple_of_tensors_to_tensor(tuple_of_tensors):
        return  torch.stack(list(tuple_of_tensors), dim=0) 
    
    def patching(batch):
        final = []
        batch = batch.unfold(2, 14, 14).unfold(3, 14, 14)
        for x in batch:
          p = x.flatten()
          p = torch.split(p,196)
          p = tuple_of_tensors_to_tensor(p)
          p.shape
          final.append(p)
        final = torch.stack(final)
        return final
    
    def forward(self, x):
        #STEM
        x = patching(x)
        first = self.Linear1(x)
        first = self.relu(first)

        #FIRST BLOCK
        second = self.Linear2(first)
        relu2 = self.relu(second)
        third = self.Linear3(relu2) #FIRST MLP

        third = torch.transpose(third,1,2)
        #print(third.shape)
        fourth = self.Linear4(third)
        relu4 = self.relu(fourth)
        fifth = self.Linear5(fourth)#SECOND MLP

        fifth = self.relu(fifth)
        #print(fifth.shape)
        sixth = self.Linear6(fifth)
        #sixth = self.relu(sixth)
        seventh = self.Linear7(sixth)


        #print(fifth.shape)
        #sixth = self.Linear6(fifth)
        #relu
        #linear
        out = sixth
        #print(out.shape)
        out = out.mean(axis=1)
        #print(out.shape)
        return out

In [15]:
# Model instantiation and initialisation 
def init_weights(m):
    if type(m) == nn.Linear: # by checking type we can init different layers in different ways
        torch.nn.init.normal_(m.weight, std=0.01)
        torch.nn.init.zeros_(m.bias)

In [16]:
# Create and initialize your model here:
num_inputs, num_hidden, num_outputs = 196, 256, 10
net = Net(num_inputs, num_outputs)

# Initialise your model here using init_weights function. If not done, default initiliazation will be used.
net.apply(init_weights)

Net(
  (Linear1): Linear(in_features=196, out_features=196, bias=True)
  (relu): ReLU()
  (elu): ELU(alpha=1.0)
  (Linear2): Linear(in_features=196, out_features=98, bias=True)
  (Linear3): Linear(in_features=98, out_features=49, bias=True)
  (Linear4): Linear(in_features=4, out_features=50, bias=True)
  (Linear5): Linear(in_features=50, out_features=100, bias=True)
  (Linear6): Linear(in_features=100, out_features=25, bias=True)
  (Linear7): Linear(in_features=25, out_features=10, bias=True)
)

# Loss and Optimization Algorithm
* As in Softmax Regression.

In [17]:
# Creare your loss here. Use Cross Entropy loss:
loss = nn.CrossEntropyLoss()
lr, wd = 0.01, 0.0005
# Creare your optimizer here. Use SGD with weight decay wd and learning rate lr.
optimizer = torch.optim.Adam(net.parameters(),weight_decay=wd,lr=lr)

# Training

* Use `my_utils.train_ch3` as in Softmax Regression

In [ ]:
num_epochs = 60
mu.train_ch3(net, train_iter, test_iter, loss, num_epochs, optimizer)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## Further Tasks

* Explore the following training options and observe their impact on the evolution of the **training loss**, the **training accuracy** and the **validation accuracy**. Different choices might have different impact on both accuracy and convergence:
    1. Change the number of hidden layers to 256
    1. Investigate different learning rates. Use 0.5, 0.9 and 0.01.
    1. Investigate adding weight decay wd=0.0005
    1. Investigate `Sigmoid` activation function. 
    1. Try different schemes for initializing the weights. 